# Find most relevant saildrone data to compare to nearby CTD data

## Basic workflow
- choose a CTD location
- determine a time-window of relevance around when the CTD was deployed to use to match to Saildrone data
- find relevant saildrone data in that timewindow.  Determine distance to ctd location (use subsampled data to speed this process up)
- determine if distance between two platforms is close enough for comparison
- if data exists in proper time range and distance range, compare instruments
- repeat for all ctd casts and all saildrones

In [94]:
#standard python import of libraries needed (pandas and datetime)
import pandas as pd
import datetime

the following code is a generic and simple *great circle* calculator and can be found at [https://github.com/CodeDrome/great-circle-distances-python](https://github.com/CodeDrome/great-circle-distances-python)

In [95]:
import math

DEGREES_IN_RADIAN = 57.29577951
MEAN_EARTH_RADIUS_KM = 6371
KILOMETRES_IN_MILE = 1.60934


class GreatCircle(object):

    """
    This class has attributes for the names and locations of a pair of cities,
    and for their distances.
    After the city attributes are set, call calculate to set the distance attributes.
    Validation is carried out and the valid attributes set. This should be checked before using
    calculated attributes.
    """

    def __init__(self):

        """
        Create a set of attributes with default values.
        """

        self.name1 = None
        self.latitude1_degrees = 0
        self.longitude1_degrees = 0
        self.latitude1_radians = 0
        self.longitude1_radians = 0

        self.name2 = None
        self.latitude2_degrees = 0
        self.longitude2_degrees = 0
        self.latitude2_radians = 0
        self.longitude2_radians = 0

        self.central_angle_radians = 0
        self.central_angle_degrees = 0
        self.distance_kilometres = 0
        self.distance_miles = 0
        self.valid = False

    def calculate(self):

        """
        Central method to set calculated attributes, which it
        does by calling other private functions.
        """

        self.__validate_degrees()

        if self.valid:
            self.__calculate_radians()
            self.__calculate_central_angle()
            self.__calculate_distance()

    def __validate_degrees(self):

        """
        Check latitudes and longitudes are within valid ranges,
        setting the valid attribute accordingly.
        """

        self.valid = True

        if self.latitude1_degrees < -90.0 or self.latitude1_degrees > 90.0:
            self.valid = False

        if self.longitude1_degrees < -180.0 or self.longitude1_degrees > 180.0:
            self.valid = False

        if self.latitude2_degrees < -90.0 or self.latitude2_degrees > 90.0:
            self.valid = False

        if self.longitude2_degrees < -180.0 or self.longitude2_degrees > 180.0:
            self.valid = False

    def __calculate_radians(self):

        """
        Calculate radians from degrees by dividing by constant.
        """

        self.latitude1_radians = self.latitude1_degrees / DEGREES_IN_RADIAN
        self.longitude1_radians = self.longitude1_degrees / DEGREES_IN_RADIAN

        self.latitude2_radians = self.latitude2_degrees / DEGREES_IN_RADIAN
        self.longitude2_radians = self.longitude2_degrees / DEGREES_IN_RADIAN

    def __calculate_central_angle(self):

        """
        Slightly complex formula for calculating the central angle
        between two points on the surface of a sphere.
        """

        if self.longitude1_radians > self.longitude2_radians:
            longitudes_abs_diff = self.longitude1_radians - self.longitude2_radians
        else:
            longitudes_abs_diff = self.longitude2_radians - self.longitude1_radians

        self.central_angle_radians = math.acos( math.sin(self.latitude1_radians)
                                         * math.sin(self.latitude2_radians)
                                         + math.cos(self.latitude1_radians)
                                         * math.cos(self.latitude2_radians)
                                         * math.cos(longitudes_abs_diff))

        self.central_angle_degrees = self.central_angle_radians * DEGREES_IN_RADIAN

    def __calculate_distance(self):

        """
        Because we are using radians, this is a simple formula multiplying the radius
        by the angle, the actual units used being irrelevant.
        Also the distance in miles is calculated from kilometres.
        """

        self.distance_kilometres = MEAN_EARTH_RADIUS_KM * self.central_angle_radians

        self.distance_miles = self.distance_kilometres / KILOMETRES_IN_MILE

In [96]:
#read in saildrone data via pandas - use subsampled data to spead up distance calculations
sd_df = pd.read_csv('C:\\Users\\saman\\OneDrive\\Documents\\NOAA\\sd-1033_data (5)_6hr.csv',parse_dates=True,index_col='TIM')
sd_df.sample()

,cruise,depth,wind gust,time text,time final,LATITUDE,LONGITUDE,TEMP_AIR_MEAN,TEMP_AIR_STDDEV,PAR_AIR_MEAN,...,TEMP_SBE37_MEAN_QC,SAL_SBE37_MEAN_QC,CHLOR_WETLABS_MEAN_QC,CHLOR_RBR_MEAN_QC,O2_CONC_AANDERAA_MEAN_QC,O2_SAT_AANDERAA_MEAN_QC,O2_CONC_SBE37_MEAN_QC,O2_SAT_SBE37_MEAN_QC,O2_CONC_RBR_MEAN_QC,O2_SAT_RBR_MEAN_QC
TIM,,,,,,,,,,,,,,,,,,,,,
2019-06-06,1033_6,1,-1.000000e+34,2019-06-06T00:00:00,2019-06-06T00:00:00,64.89807,-168.906,1.83,0.01,1577.0,...,1.1996,32.4958,1.25,1.146609,410.28,116.81,408.9,115.87,411.61,116.5


In [97]:
#read in healy data via pandas
he1901_df = pd.read_excel('C:\\Users\\saman\\Downloads\\CTD_dy1908_final1.xlsx',sheet_name=0,skiprows=34,parse_dates=True)

***ONCE YOU KNOW WHATS GOING ON, YOU CAN DELETE THE CELLS FROM HERE TO WHERE I SAY KEEP CELLS***

Lets just start with *CTD001* from the healy1901 cruise.  I don't know if there is a valid saildrone point nearby but it will show the math

We can use pandas to only retrieve the CTD of interest by subsetting on name/value pairs.  Its a pandas thing and pretty easy to pick up on.

In [98]:
he1901_df[he1901_df['Cast'] == 1] #this is pandas speak, and only one way to do it, to get all casts that equal 1 and index on those rows only

,Cruise,Cast,Latitude,Longitude,Date Time,P_4,BTL_103,S_42,S_41,ST_70,...,wind_direction,wind_speed,air_temperature,Station_Name,profile_id,SI_188,PO4_186,NO2_184,NO3_182,NH4_189
0,dy1908,1,56.766834,195.66083,2019-09-20 10:05:40,1,NaN,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
1,dy1909,1,56.766834,195.66083,2019-09-20 10:05:40,2,7.0,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,6.6,0.198,0.02,0.2,0.1
2,dy1910,1,56.766834,195.66083,2019-09-20 10:05:40,2,8.0,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
3,dy1911,1,56.766834,195.66083,2019-09-20 10:05:40,3,NaN,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
4,dy1912,1,56.766834,195.66083,2019-09-20 10:05:40,4,NaN,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,dy1974,1,56.766834,195.66083,2019-09-20 10:05:40,66,NaN,31.9244,31.9244,25.273193,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
67,dy1975,1,56.766834,195.66083,2019-09-20 10:05:40,67,NaN,31.9244,31.9244,25.273315,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
68,dy1976,1,56.766834,195.66083,2019-09-20 10:05:40,68,NaN,31.9244,31.9244,25.273315,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN
69,dy1977,1,56.766834,195.66083,2019-09-20 10:05:40,69,NaN,31.9244,31.9244,25.273438,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN


The results above are the entire profile, you are only interested in near surface so you could also say that P_4 must equal some value: see following cell

In [99]:
he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == 4]

<ipython-input-99-51847829b33a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == 4]


,Cruise,Cast,Latitude,Longitude,Date Time,P_4,BTL_103,S_42,S_41,ST_70,...,wind_direction,wind_speed,air_temperature,Station_Name,profile_id,SI_188,PO4_186,NO2_184,NO3_182,NH4_189
4,dy1912,1,56.766834,195.66083,2019-09-20 10:05:40,4,NaN,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN


or, without knowing the depth, but just wanting the shallowest (minimum pressure... as the .min() function on the 'P_4' column will do) you could say:

In [100]:
he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == he1901_df[he1901_df['Cast'] == 1]['P_4'].min()] #i'm stringing together index calls here and strongly suggest you look at pandas/python examples for how to index dataframes if the above is confusing.  

<ipython-input-100-d5965d1aa1f9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == he1901_df[he1901_df['Cast'] == 1]['P_4'].min()] #i'm stringing together index calls here and strongly suggest you look at pandas/python examples for how to index dataframes if the above is confusing.


,Cruise,Cast,Latitude,Longitude,Date Time,P_4,BTL_103,S_42,S_41,ST_70,...,wind_direction,wind_speed,air_temperature,Station_Name,profile_id,SI_188,PO4_186,NO2_184,NO3_182,NH4_189
0,dy1908,1,56.766834,195.66083,2019-09-20 10:05:40,1,NaN,31.6482,31.6507,24.014404,...,NaN,NaN,NaN,NaN,dy1908c001,NaN,NaN,NaN,NaN,NaN


If you do the above, looping over all ctd casts, you will get a new list of only the shallowest datapoints in each cast... but perhaps you already did this in another routine... lets move on, to calculating distance

Taking the CTD001 example above, you can see I have latitude and Longitude... and same is true of the saildrone data.  Lets choose a 24hour window (+/- 12hrs) and see if there is any saildrone data in that time period

In [101]:
ctdsfc = he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == he1901_df[he1901_df['Cast'] == 1]['P_4'].min()] #save a variable for ease of calculations to follow

# this will report the datetime of the cast and will be the central point of the window we want to look for in time
# the [0] at the end chooses the array value for furthur purposes and the datetime function is used to add 12 hours and subtract 12 hours from the central time
(ctdsfc['Date Time']-datetime.timedelta(hours=12))[0]

<ipython-input-101-59c2ac1eb8c8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ctdsfc = he1901_df[he1901_df['Cast'] == 1][he1901_df['P_4'] == he1901_df[he1901_df['Cast'] == 1]['P_4'].min()] #save a variable for ease of calculations to follow


Timestamp('2019-09-19 22:05:40')

In [102]:
ctd_sd_match = sd_df[(pd.to_datetime(ctdsfc['Date Time'])-datetime.timedelta(hours=12))[0]:(pd.to_datetime(ctdsfc['Date Time'])+datetime.timedelta(hours=12))[0]]
ctd_sd_match

,cruise,depth,wind gust,time text,time final,LATITUDE,LONGITUDE,TEMP_AIR_MEAN,TEMP_AIR_STDDEV,PAR_AIR_MEAN,...,TEMP_SBE37_MEAN_QC,SAL_SBE37_MEAN_QC,CHLOR_WETLABS_MEAN_QC,CHLOR_RBR_MEAN_QC,O2_CONC_AANDERAA_MEAN_QC,O2_SAT_AANDERAA_MEAN_QC,O2_CONC_SBE37_MEAN_QC,O2_SAT_SBE37_MEAN_QC,O2_CONC_RBR_MEAN_QC,O2_SAT_RBR_MEAN_QC
TIM,,,,,,,,,,,,,,,,,,,,,
2019-09-20 00:00:00,1033_6,1,-1.000000e+34,2019-09-20T00:00:00,2019-09-20T00:00:00,66.51567,-167.2310,6.45,0.12,280.0,...,10.9998,-1.000000e+34,0.98,-1.000000e+34,260.26,92.90,-1.000000e+34,-1.000000e+34,296.82,102.40
2019-09-20 06:00:00,1033_6,1,-1.000000e+34,2019-09-20T06:00:00,2019-09-20T06:00:00,66.32318,-167.4372,6.08,0.05,0.0,...,10.6173,-1.000000e+34,1.25,-1.000000e+34,266.75,94.41,-1.000000e+34,-1.000000e+34,304.08,104.39
2019-09-20 12:00:00,1033_6,1,-1.000000e+34,2019-09-20T12:00:00,2019-09-20T12:00:00,66.12798,-167.6870,5.88,0.02,0.0,...,10.7448,-1.000000e+34,1.19,-1.000000e+34,263.41,93.49,-1.000000e+34,-1.000000e+34,289.49,99.47
2019-09-20 18:00:00,1033_6,1,-1.000000e+34,2019-09-20T18:00:00,2019-09-20T18:00:00,66.02276,-167.8513,5.37,0.08,68.0,...,10.6061,-1.000000e+34,0.92,-1.000000e+34,258.04,91.12,-1.000000e+34,-1.000000e+34,281.66,96.85


So the 4 rows shown above are the relevant time values for the saildrone data, but are they close enough? lets use the great circle calculator between each of these four points and the ctd location 

In [103]:
#defined at the beginning of this notebook, its a python method and has some unique ways to call its functions
gc = GreatCircle()

#pt 1
gc.name1 = 'HLY Cast'
gc.latitude1_degrees = ctdsfc['Latitude'][0]
gc.longitude1_degrees = ctdsfc['Longitude'][0]

#choose just the first SD matched in time location for now, python is zero indexed
gc.name1 = 'SD Location'
gc.latitude2_degrees = ctd_sd_match['LATITUDE'][0]
gc.longitude2_degrees = ctd_sd_match['LONGITUDE'][0]

gc.calculate()

In [104]:
print(f'Distance between points are {gc.distance_kilometres} kilometers')

Distance between points are 0 kilometers


In [105]:
#loop over all valid saildrone points at this time and
#defined at the beginning of this notebook, its a python method and has some unique ways to call its functions
gc = GreatCircle()

#pt 1
gc.name1 = 'HLY Cast'
gc.latitude1_degrees = ctdsfc['Latitude'][0]
gc.longitude1_degrees = ctdsfc['Longitude'][0]

for i,rows in ctd_sd_match.iterrows():
    gc.name1 = 'SD Location'
    gc.latitude2_degrees = rows['LATITUDE']
    gc.longitude2_degrees = rows['LONGITUDE']

    gc.calculate()
    print(f'Distance between points are {gc.distance_kilometres} kilometers for sd at time {i}')

Distance between points are 0 kilometers for sd at time 2019-09-20 00:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 06:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 12:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 18:00:00


***KEEP ALL CELLS BELOW***


So for the example of Saildrone 1033 and CTD001 on HLY1901 - the saildrone was ~1300km away from the the ctd in the window of 1day around the cast.

So now the goal would be to loop over all casts for this one Saildrone and see if there are any points of interest.  I'm gonna do it in one cell... this means that if you want to perform analysis's on other saildrones or cruises, you aught to be able to just change the data files and then run this cell.  **Warning** Some previous cells that are SD-1033 specific will break if you do this blindly...

**NOTE** for the saildrone data, it looks to have some information about the distance between the drones... you may be able to use this information also when looking at all drones or you may not ... up to you


In [106]:
#we are going to use another pandas trick call groupby to associate unique casts together and then loop through each cast
he1901_df.groupby('Cast').groups.keys() #will show all the unique 'Cast' numbers

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])

In [107]:
for groups in he1901_df.groupby('Cast').groups.keys():
    cast = he1901_df.groupby('Cast').get_group(groups)
    print(f'Working on CTD {groups}')
    #get relevant CTD info
    ctdsfc = he1901_df[he1901_df['Cast'] == groups][he1901_df['P_4'] == he1901_df[he1901_df['Cast'] == groups]['P_4'].min()]
    #get matching saildrone info
    ctd_sd_match = sd_df[(pd.to_datetime(ctdsfc['Date Time'])-datetime.timedelta(hours=12)).iloc[0]:(pd.to_datetime(ctdsfc['Date Time'])+datetime.timedelta(hours=12)).iloc[0]]

    #calc great circle distance
    gc = GreatCircle()

    #pt 1
    gc.name1 = 'HLY Cast'
    gc.latitude1_degrees = ctdsfc['Latitude'].iloc[0]
    gc.longitude1_degrees = ctdsfc['Longitude'].iloc[0]

    for i,rows in ctd_sd_match.iterrows():
        gc.name1 = 'SD Location'
        gc.latitude2_degrees = rows['LATITUDE']
        gc.longitude2_degrees = rows['LONGITUDE']

        gc.calculate()
        print(f'Distance between points are {gc.distance_kilometres} kilometers for sd at time {i}')


Working on CTD 1
Distance between points are 0 kilometers for sd at time 2019-09-20 00:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 06:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 12:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 18:00:00
Working on CTD 2
Distance between points are 0 kilometers for sd at time 2019-09-20 06:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 12:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 18:00:00
Distance between points are 0 kilometers for sd at time 2019-09-21 00:00:00
Working on CTD 3
Distance between points are 0 kilometers for sd at time 2019-09-20 06:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 12:00:00
Distance between points are 0 kilometers for sd at time 2019-09-20 18:00:00
Distance between points are 0 kilometers for sd at time 2019-09-21 00:00:00
Working on CTD 4
Distance between poi

The output above will present you with the distance and time of each CTD and Saildrone point.  VSC may give you an error about perfomance on the output... just click the link and add a zero to the number it provides and then rerun the cell.  Also, you are likely to see some warnings... in this case, i don't think you need to worry about any as they all have to do with "boolean indexing". any others may need to be explored.


For the example of HE1901 CTD data and SD-1033 :  CTD's 35-40 look to be the promising ones for your comparison.